# -GAN kullanrak resim renklendirme - Image Colorization using GAN 
#### Generative Adversarial Networks iki temel yapıdan oluşur. Generator (Üretici) ve Discriminator (Ayırıcı). Generator bir veri üretir ve Discriminator üretilen bu verini gerçek mi yoksa sahte mi olduğunu anlamaya çalışır. Bu geri bildirimle Disciriminator her zaman gerçek veriye yaklaşmaya çalışır ve bu sayede Generator zamanla gerçeğe yakın veriler üretmeye başlar.
#### Bu projede önce manzara resimlerinin bulunduğu verisetindeki görsellerin bir kısmı L*a*b formatında siyah beyaz hale dönüşütürlür. Generator bu şekilde yeni görseller üretmeye çalışır. Discirminator elindeki renkli verilerle Generator tarafınfan üretilen yeni resimleri karşılaştırır. Ve renkli resimşler göre hata hesabı yapar. Bu sayede bir süre sonra Generator siyah beyaz resimlerin renklerini tahmin etmeye başlar ve gerçeğe yakın sonuçlar üretir.

In [ ]:
import os
import numpy as np
from PIL import Image
from skimage.color import rgb2lab, lab2rgb
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #cuda aktifleştirme



In [ ]:
#veriseti işlemleri
class ColorizationDataset(Dataset):  
    def __init__(self, paths, split='train'): #veri setinin train kısmı alınır
        self.splits = split
        self.paths = paths
        self.transforms = transforms.Compose([ #transforms modülü ile 256x256 olarak boyutlandırılır
            transforms.Resize((256, 256)),
            transforms.RandomHorizontalFlip(), #resimler yatay olarak ters çevrilir
        ])

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB") #resimler önce rgb formatına dönüştürülür
        img = self.transforms(img)
        img = np.array(img)
        img_lab = rgb2lab(img).astype("float32") #rgb formatından lab formantına (l: siyah, ab:renkli kısım) dönüştürülür
        img_lab = transforms.ToTensor()(img_lab)
        L = img_lab[[0], ...] / 50. - 1.
        ab = img_lab[[1, 2], ...] / 110.
        return {'L': L, 'ab': ab}

    def __len__(self):
        return len(self.paths)

In [ ]:
path = r"C:\Users\azsar\Desktop\manzaralar" #dosya yolu
resimler = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')] #resimleri okuma
train_paths, val_paths = train_test_split(resimler, test_size=0.2, random_state=99) #verisetini train ve test olarak ayırma


In [ ]:
train_dataloader = DataLoader(ColorizationDataset(train_paths), batch_size=16, shuffle=True) #train kısmını yüklme (16 şar batch olarak)
test_dataloader = DataLoader(ColorizationDataset(val_paths, split='val'), batch_size=16, shuffle=True) #test kısmını yükleme

In [ ]:
class Generator(nn.Module): #GAN yapısının generator (resim üretici) kısmı 
    def __init__(self, input_c=1, output_c=2, num_filters=64):
        super().__init__()
        self.model = nn.Sequential( #konvolüsyon katmanları ileri ve geri konvolüsyon işlmei yapılıyoır
            #konvolüsyon
            nn.Conv2d(input_c, num_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters),
            nn.LeakyReLU(0.2, True),
            
            nn.Conv2d(num_filters, num_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters*2),
            nn.LeakyReLU(0.2, True),
            
            nn.Conv2d(num_filters*2, num_filters*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters*4),
            nn.LeakyReLU(0.2, True),
            
            nn.Conv2d(num_filters*4, num_filters*8, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters*8),
            nn.LeakyReLU(0.2, True),

            #ters konvolüsyon
            nn.ConvTranspose2d(num_filters*8, num_filters*4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters*4),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(num_filters*4, num_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters*2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(num_filters*2, num_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(num_filters),
            nn.ReLU(True),

            nn.ConvTranspose2d(num_filters, output_c, kernel_size=4, stride=2, padding=1), 
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

#GAN yapsının discriminator (ayırıcı-gerçek sahte tespit edici) kısmı
class Discriminator(nn.Module):
    def __init__(self, input_c=3, num_filters=64, n_down=3):
        super().__init__()
        model = [self.get_layers(input_c, num_filters, norm=False)]
        model += [self.get_layers(num_filters * 2 ** i, num_filters * 2 ** (i + 1), s=1 if i == (n_down-1) else 2) 
                  for i in range(n_down)]
        model += [self.get_layers(num_filters * 2 ** n_down, 1, s=1, norm=False, act=False)]
        self.model = nn.Sequential(*model)

    def get_layers(self, ni, nf, k=4, s=2, p=1, norm=True, act=True):
        layers = [nn.Conv2d(ni, nf, k, s, p, bias=not norm)]
        if norm: layers += [nn.BatchNorm2d(nf)]
        if act: layers += [nn.LeakyReLU(0.2, True)]
        return nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

  

In [ ]:
class GANLoss(nn.Module): #GAN doğruluk değeri hesaplama işlemleri
    def __init__(self, gan_mode='vanilla', real_label=1.0, fake_label=0.0):
        super().__init__()
        self.register_buffer('real_label', torch.tensor(real_label)) #discriminator'ün fake-real tanımlaması
        self.register_buffer('fake_label', torch.tensor(fake_label))
        
        self.loss = nn.BCEWithLogitsLoss() #loss fonksiyonu
        
    def get_labels(self, preds, target_is_real):
        if target_is_real:
            labels = self.real_label
        else:
            labels = self.fake_label
        return labels.expand_as(preds)

    def __call__(self, preds, target_is_real):
        labels = self.get_labels(preds, target_is_real)
        loss = self.loss(preds, labels)
        return loss


In [ ]:
class MainModel(nn.Module): #GAN'ın çalıştırıldığı bölüm
    def __init__(self, net_G=None, lr_G=2e-4, lr_D=2e-4, 
                 beta1=0.5, beta2=0.999, lambda_L1=100.):
        super().__init__()
        self.device = device
        self.lambda_L1 = lambda_L1
        
        if net_G is None:
            self.net_G = Generator(input_c=1, output_c=2, num_filters=64)
        else:
            self.net_G = net_G
        self.net_D = Discriminator(input_c=3, n_down=3, num_filters=64)
        self.GANcriterion = GANLoss(gan_mode='vanilla')
        self.L1criterion = nn.L1Loss()
        self.opt_G = optim.Adam(self.net_G.parameters(), lr=lr_G, betas=(beta1, beta2))
        self.opt_D = optim.Adam(self.net_D.parameters(), lr=lr_D, betas=(beta1, beta2))
        
        self.net_G.to(device)
        self.net_D.to(device)
        self.GANcriterion.to(device)
    
    def set_input(self, data):
        self.L = data['L'].to(self.device)   #input olarak L:siyah beyaz görüntü
        self.ab = data['ab'].to(self.device) #ab renkli görüntü
    
    def forward(self):
        self.fake_color = self.net_G(self.L)
    
    def backward_D(self):
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image.detach())
        self.loss_D_fake = self.GANcriterion(fake_preds, False)
        real_image = torch.cat([self.L, self.ab], dim=1)
        real_preds = self.net_D(real_image)
        self.loss_D_real = self.GANcriterion(real_preds, True)
        self.loss_D = (self.loss_D_fake + self.loss_D_real) * 0.5
        self.loss_D.backward()
    
    def backward_G(self):
        fake_image = torch.cat([self.L, self.fake_color], dim=1)
        fake_preds = self.net_D(fake_image)
        self.loss_G_GAN = self.GANcriterion(fake_preds, True)
        self.loss_G_L1 = self.L1criterion(self.fake_color, self.ab) * self.lambda_L1
        self.loss_G = self.loss_G_GAN + self.loss_G_L1
        self.loss_G.backward()
    
    def optimize(self):
        self.forward()
        self.net_D.train()
        self.opt_D.zero_grad()
        self.backward_D()
        self.opt_D.step()
        
        self.net_G.train()
        self.opt_G.zero_grad()
        self.backward_G()
        self.opt_G.step()

        

In [ ]:
#çıktıların görselleştirme kısmı
def visualize(model, data):
    model.net_G.eval()
    
    with torch.no_grad():
        model.set_input(data)
        model.forward()
    model.net_G.train()
    fake_color = model.fake_color.detach()
    real_color = model.ab
    L = model.L
    fake_imgs = lab_to_rgb(L, fake_color) #fake resimleri renklendirme
    real_imgs = lab_to_rgb(L, real_color) #real resimleri renklendirme
    
    fig = plt.figure(figsize=(15, 8))
    for i in range(5):
        ax = plt.subplot(3, 5, i + 1)
        ax.imshow(L[i][0].cpu(), cmap='gray')
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 5)
        ax.imshow(fake_imgs[i])
        ax.axis("off")
        ax = plt.subplot(3, 5, i + 1 + 10)
        ax.imshow(real_imgs[i])
        ax.axis("off")
    plt.show()

def lab_to_rgb(L, ab): #siyah beyaz için oluşturulan lab görüntülerin rgb renkli görüntülere çevrilmesi
    L = (L + 1.) * 50.
    ab = ab * 110.
    Lab = torch.cat([L, ab], dim=1).permute(0, 2, 3, 1).cpu().numpy()
    rgb_imgs = []
    for img in Lab:
        img_rgb = lab2rgb(img)
        rgb_imgs.append(img_rgb)
    return np.stack(rgb_imgs, axis=0)


In [ ]:
#eğitim kısmı
def train_model(model, train_dataloader, epochs, display_every=30):
    data = next(iter(train_dataloader))
    for e in range(epochs):
        loss_meter_dict = {'loss_D': [], 'loss_G': [], 'loss_G_GAN': [], 'loss_G_L1': []}
        for i, data in enumerate(tqdm(train_dataloader)):
            model.set_input(data) 
            model.optimize()
            
            # kayıp değerlerini kaydetme
            loss_meter_dict['loss_D'].append(model.loss_D.item())
            loss_meter_dict['loss_G'].append(model.loss_G.item())
            loss_meter_dict['loss_G_GAN'].append(model.loss_G_GAN.item())
            loss_meter_dict['loss_G_L1'].append(model.loss_G_L1.item())
            
            if i % display_every == 0:
                print(f"\nEpoch {e+1}/{epochs}")
                print(f"Iteration {i}/{len(train_dataloader)}")
                print(f"loss_D: {np.mean(loss_meter_dict['loss_D']):.5f}")
                print(f"loss_G: {np.mean(loss_meter_dict['loss_G']):.5f}")
                print(f"loss_G_GAN: {np.mean(loss_meter_dict['loss_G_GAN']):.5f}")
                print(f"loss_G_L1: {np.mean(loss_meter_dict['loss_G_L1']):.5f}")
                visualize(model, data)

In [ ]:

model = MainModel()
train_model(model, train_dataloader, epochs=50, display_every=30) #50 epoch ve 50 iterasyonda bir sonuç gösterecek modeli şekilde eğitme
